In [1]:
!pip install pyautogui

     |████████████████████████████████| 59 kB 157 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 132 kB 444 kB/s eta 0:00:01
  Created wheel for pyautogui: filename=PyAutoGUI-0.9.53-py3-none-any.whl size=36583 sha256=165768d9a1305f06f36561eef19940ce58fae0d1adb6c1e85e28e73dc0d2e95c
  Stored in directory: /home/mohamed/.cache/pip/wheels/54/04/f7/55704a9d374ed923785a4fdc0ef00151fa25306b3b93345532
  Created wheel for pygetwindow: filename=PyGetWindow-0.0.9-py3-none-any.whl size=11078 sha256=08c6a2946272fb0c776e27e9a74c2afd735d94b6351766c9c7b0bd15d12321d3
  Stored in directory: /home/mohamed/.cache/pip/wheels/91/e3/04/184bda98ec7f7d5f0ba1f372bcf7b3ba86325151ae1dd690fe
  Created wheel for pyscreeze: filename=PyScreeze-0.1.28-py3-none-

In [3]:
!pip install mouse

In [6]:
!pip install mediapipe

     |████████████████████████████████| 32.7 MB 1.7 kB/s eta 0:00:01     |██████████▎                     | 10.6 MB 190 kB/s eta 0:01:57     |█████████████████               | 17.3 MB 152 kB/s eta 0:01:42     |█████████████████████████▉      | 26.5 MB 127 kB/s eta 0:00:50     |██████████████████████████▊     | 27.3 MB 299 kB/s eta 0:00:19


In [5]:
!pip install autopy

     |████████████████████████████████| 8.3 MB 21 kB/s  eta 0:00:01     |████████████████▉               | 4.4 MB 1.5 MB/s eta 0:00:03


In [12]:
import cv2
import numpy as np
import sys
import mouse
#sys.path.insert(1, 'F:/machine learning/modules/')
import handLandmarkMP as hlm
import pyautogui
import time
import autopy

In [13]:
screen_width, screen_height= pyautogui.size()

In [14]:
print("width: {}, height: {}".format(screen_width,screen_height))

width: 1366, height: 768


In [15]:
screen_height//3

256

In [16]:
hands = hlm.HandDetector()

In [17]:
img_center = (img.shape[1]//2,img.shape[0]//2)
x1,y1 = img_center[0] - (455//2), img_center[1] - (256//2)
x2,y2 = img_center[0] + (455//2), img_center[1] + (256//2)
venv_rec_x = x2 - x1
venv_rec_y = y2 - y1
venv_rec_y

NameError: name 'img' is not defined

In [18]:
np.isclose([11, 9], [0, 0],atol = 10)

array([False,  True])

In [27]:
cap = cv2.VideoCapture(0)
success, img = cap.read()
img_center = (img.shape[1]//2,img.shape[0]//2)
x1,y1 = img_center[0] - (455//2), img_center[1] - (256//2)
x2,y2 = img_center[0] + (455//2), img_center[1] + (256//2)
a,b,c,d = x1,y1,x2,y2
e,f,g,h = 1,1,1366,768

start = time.time()

avg_factor = 7
avg_index = 0
history_x = np.ones(avg_factor) * 1366
history_y = np.ones(avg_factor) * 768
x_before = 1366 // 2
y_before = 768 // 2

start_point = (x1,y1)
end_point = (x2,y2)
rectangle_color = (155,155,155)
rectangle_thickness = 3
frames = 0

font = cv2.FONT_HERSHEY_SIMPLEX

while cap.isOpened():
    success, img = cap.read()
    #img = cv2.resize(img,(screen_width//2,screen_height//2))
    img = cv2.flip(img, 1)
    if not success:
        print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
        continue
    img = hands.findHands(img)
    hands_positions = hands.findPositions(img)
    #print(hands_positions)
    #mouse.move(screen_width//2, screen_height//2, absolute=False, duration=0)
    img = cv2.rectangle(img, start_point, end_point, rectangle_color, rectangle_thickness)
    if hands_positions:
        index = np.array(hands_positions['hand1']['index'][-1])
        middle = np.array(hands_positions['hand1']['middle'][-1])
        ring = np.array(hands_positions['hand1']['ring'][-1])
        #print(f"middle co-ordinates : {middle}")
        #print(f"index co-ordinates : {index}")
        
        # get the ecludian distance between middle finger tip and index finger tip
        x, y = np.abs(middle-index)
        dist = int((x**2 + y**2)**0.5)
        print(dist)
        
        #print(f"index pos:{index} middle pos {middle}")
        
        # map x and y values from the frame to the screen size
        x_ = e + (middle[0] - a) * (g - e) / (c - a)
        y_ = f + (middle[1] - b) * (h - f) / (d - b)
        
        if (np.isclose(dist,60,atol = 7)):
            autopy.mouse.click()
        
        if (np.all(np.isclose([x_,y_],[x_before,y_before],atol = 10))):
            x_ = x_before
            y_ = y_before
        
        # appending current average
        history_x[avg_index] = int(x_)
        history_y[avg_index] = int(y_)
        
        # averaging over position history
        pos_avg_x = int(np.average(history_x))
        pos_avg_y = int(np.average(history_y))
        
        
        #print(history_x)
        
        mouse.move(pos_avg_x, pos_avg_y, absolute=True, duration=0)
        #cv2.circle(img, hands_positions['hand1']['middle'][-1], 0, (0, 0, 0), 6)
        
        x_before = x_
        y_before = y_
    
    
    end = time.time()
    avg_index += 1
    
    if (avg_index / avg_factor == 1):
        avg_index = 0
    fps = str(int(frames / (end-start)))
    frames += 1
    cv2.putText(img, fps, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)
    cv2.imshow("Image", img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

9
32
25
28
30
29
29
28
27
27
27
26
27
27
26
28
27
28
28
28
27
27
28
27
26
26
26
26
26
26
28
26
27
27
27
27
27
27
28
30
29
30
29
30
28
30
31
30
30
29
30
30
28
27
28
29
29
27
29
29
30
29
29
29
29
28
29
28
29
30
28
29
29
36
53
58
55
47
37
32
31
31
32
32
33
35
53
58
53
43
40
37
34
33
32
33
31
33
34
33
35
34
35
33
32
34
34
34
33
33
32
33
33
32
32
31
32
33
31
32
32
32
31
31
31
32
32
32
32
33
33
31
33
31
32
32
32
32
32
32
31
31
31
30
31
31
32
37
61
68
62
58
43
36
35
36
35
34
32
32
31
30
30
29
30
31
31
31
30
29
28
25
23
22
24
23
22
23
22
22
21
28
49
54
49
32
27
26
24
25
26
26
26
26
27
27
26
24
26
44
23
23
41
35
27
30
31
32
31
31
31
29
27
24
24
29
26
23
53
35
26
23
63
43
36
36
31
32
32
33
34
34
34
33
34
34
33
34
33
30
30
31
26
25
23
22
23
22
21
19
18
16
16
16
16
14
13
13
15
18
21
21
22
20
22
22
22
22
21
22
22
21
20
22
19
20
20
19
20
18
18
17
18
17
19
19
19
18
19
19
18
18
17
16
19
19
19
23
23
26
27
27
27
28
26
26
27
27
26
27
27
27
27
27
27
27
27
28
28
28
28
27
29
27
29
28
28
27
27
27
28
27
27
27

12
12
11
9
13
9
11
9
16
12
12
10
13
13
8
10
11
9
7
6
6
6
6
7
8
6
10
9
7
8
9
9
7
7
9
9
9
10
10
10
9
9
8
6
6
6
5
7
7
35
32


In [21]:
mouse.move(1366, 640, absolute=True, duration=0.1)